In [1]:
from inspect import signature
import logging

# Metaclass to reject mixed-case method names
class NoMixedCaseMeta(type):
    def __new__(cls, clsname, bases, clsdict):
        for name in clsdict:
            if name.lower() != name:
                raise TypeError(f"Bad attribute name: {name}")
        return super().__new__(cls, clsname, bases, clsdict)

# Root class using NoMixedCaseMeta
class Root(metaclass=NoMixedCaseMeta):
    pass

# Valid class
class A(Root):
    def foo_bar(self):  # OK
        pass

# Invalid class (raises TypeError)
try:
    class B(Root):
        def fooBar(self):  # TypeError: Bad attribute name: fooBar
            pass
except TypeError as e:
    print(e)

# Metaclass to enforce method signature consistency
class MatchSignaturesMeta(type):
    def __init__(self, clsname, bases, clsdict):
        super().__init__(clsname, bases, clsdict)
        sup = super(self, self)  # Get the parent class

        for name, value in clsdict.items():
            if name.startswith('_') or not callable(value):
                continue

            prev_dfn = getattr(sup, name, None)
            if prev_dfn:
                prev_sig = signature(prev_dfn)
                val_sig = signature(value)
                if prev_sig != val_sig:
                    logging.warning("Signature mismatch in %s. %s != %s",
                                    value.__qualname__, prev_sig, val_sig)

# Root class using MatchSignaturesMeta
class RootSignatures(metaclass=MatchSignaturesMeta):
    pass

class A(RootSignatures):
    def foo(self, x, y):
        pass

    def spam(self, x, *, z):
        pass

# Class with method signature mismatches
class B(A):
    def foo(self, a, b):  # Signature mismatch warning
        pass

    def spam(self, x, z):  # Signature mismatch warning
        pass


Bad attribute name: fooBar
